In [ ]:
import pandas as pd
import numpy as np

In [1]:
from meteostat import Stations

stations = Stations()

#The coordinates below represent the center of the Bay Area

stations = stations.nearby(37.6965, -122.1158)

wc_stations = stations.fetch(1000)

                                             name country region    wmo  icao  \
id                                                                              
72585                      Hayward / Russell City      US     CA  72585  KHWD   
72493        Metro Oakland International  Airport      US     CA  72493  KOAK   
74506                   Alameda Naval Air Station      US     CA  74506  KNGZ   
KSQL0  San Carlos / Silver Penny Mobile Home Park      US     CA   <NA>  KSQL   
72494                       San Francisco Airport      US     CA  72494  KSFO   
...                                           ...     ...    ...    ...   ...   
XPC39                                Buffalo AGCM      CA     AB   <NA>  <NA>   
KAIA0            Alliance / Meadows Trailer Court      US     NE   <NA>  KAIA   
72768               Glasgow International Airport      US     MT  72768  KGGW   
KSPD0                                 Springfield      US     CO   <NA>  KSPD   
71888                       